# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import re
import nltk
import time
import pickle
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sqlalchemy import create_engine, text

from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db', echo=True)
connection = engine.connect()

# Execute SQL query to select table and create df
df = pd.read_sql_query("SELECT * FROM ETL_Messages", connection)

# Close the database connection
connection.close()

# messages column
X = df.message.values

# categorical col
Y = df[df.columns[4:]] # skip the first three columns

2025-01-05 15:10:44,950 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-05 15:10:44,950 INFO sqlalchemy.engine.Engine SELECT * FROM ETL_Messages
2025-01-05 15:10:44,950 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-05 15:10:45,468 INFO sqlalchemy.engine.Engine ROLLBACK


In [3]:
# check result
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [5]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [6]:
# test the tokenize function
tokenize(X[5])

['information', 'national', 'palace']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
# Create a Classifier
# dt_classifier = DecisionTreeClassifier(random_state=42)

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=5)))
    ])

In [16]:
# read parameters from ML pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001D95873CDC0>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)))],
 'transform_input': None,
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001D95873CDC0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Fit the model (measure time just for information)
start_fit_pipeline = time.time()

pipeline.fit(X_train, y_train)

end_fit_pipeline = time.time()
measured_time = end_fit_pipeline - start_fit_pipeline
print(f"time elapsed {measured_time}s")

c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


time elapsed 114.1592628955841s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
y_pred = pipeline.predict(X_test)

In [19]:
for i, col in enumerate(y_test.columns):
    print(f'message category: {col}\n')
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

message category: related

              precision    recall  f1-score   support

           0       0.64      0.47      0.54      1266
           1       0.84      0.91      0.87      3938
           2       0.57      0.20      0.30        40

    accuracy                           0.80      5244
   macro avg       0.68      0.53      0.57      5244
weighted avg       0.79      0.80      0.79      5244

message category: request

              precision    recall  f1-score   support

           0       0.89      0.98      0.93      4349
           1       0.80      0.43      0.56       895

    accuracy                           0.88      5244
   macro avg       0.84      0.71      0.75      5244
weighted avg       0.88      0.88      0.87      5244

message category: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      

c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      5073
           1       0.50      0.02      0.04       171

    accuracy                           0.97      5244
   macro avg       0.73      0.51      0.51      5244
weighted avg       0.95      0.97      0.95      5244

message category: death

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5007
           1       0.85      0.12      0.21       237

    accuracy                           0.96      5244
   macro avg       0.90      0.56      0.59      5244
weighted avg       0.95      0.96      0.94      5244

message category: other_aid

              precision    recall  f1-score   support

           0       0.87      0.99      0.93      4549
           1       0.46      0.04      0.08       695

    accuracy                           0.87      5244
   macro avg       0.67      0.52      0.50      5244
weighted avg       0

c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
parameters = {
        'clf__estimator__n_estimators': [10, 20],
        'vect__ngram_range': [(1, 1), (1,2)]
}

# pipeline.fit(X_train,y_train)

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train,y_train)

print("Grid Search Results (best found parameter):",cv.best_params_)

c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\

Grid Search Results (best found parameter): {'clf__estimator__n_estimators': 20, 'vect__ngram_range': (1, 2)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
y_pred = cv.predict(X_test)

In [25]:
for i, col in enumerate(y_test.columns):
    print(f'message category: {col}\n')
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

message category: related

              precision    recall  f1-score   support

           0       0.63      0.47      0.54      1266
           1       0.84      0.91      0.87      3938
           2       0.44      0.28      0.34        40

    accuracy                           0.80      5244
   macro avg       0.64      0.55      0.58      5244
weighted avg       0.79      0.80      0.79      5244

message category: request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.82      0.45      0.58       895

    accuracy                           0.89      5244
   macro avg       0.86      0.72      0.76      5244
weighted avg       0.88      0.89      0.88      5244

message category: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      

c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5184
           1       1.00      0.02      0.03        60

    accuracy                           0.99      5244
   macro avg       0.99      0.51      0.51      5244
weighted avg       0.99      0.99      0.98      5244

message category: refugees

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      5073
           1       0.57      0.02      0.04       171

    accuracy                           0.97      5244
   macro avg       0.77      0.51      0.51      5244
weighted avg       0.96      0.97      0.95      5244

message category: death

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5007
           1       0.90      0.16      0.27       237

    accuracy                           0.96      5244
   macro avg       0.93      0.58      0.63      5244
weighted avg       0.

c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\users\daniel\pycharmprojects\disasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# TBD

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv2, open('classifier.pkl','wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.